In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
train = pd.read_csv('../input/train.tsv',delimiter='\t')
test = pd.read_csv('../input/test.tsv',delimiter='\t')
sample = pd.read_csv('../input/sample_submission.csv')

In [ ]:
y_train = train.price.values
test_id = test.test_id
train = train.drop("train_id", axis = 1)
test = test.drop("test_id", axis = 1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [ ]:
all_data = pd.concat((train, test)).reset_index(drop = True)

In [ ]:
all_data= count_vect.fit_transform(all_data['name'])

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50)
all_data = svd.fit_transform(all_data)

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

model_lgb = lgb.LGBMRegressor(boosting_type='gbdt', num_leaves=80, max_depth=8, learning_rate=0.05, n_estimators=900, max_bin=255, 
                       seed=42, nthread=-1, silent=True)
#model_lgb = lgb.LGBMRegressor (objective= 'regression', num_leaves = 5,learning_rate = 0.05, n_estimators = 720,
#                                                          max_bin = 55, bagging_fraction = 0.2319, feature_fraction_seed = 9, bagging_seed = 9,
 #                                                         min_data_in_leaf = 6, min_sum_hessian_in_leaf = 11)

In [ ]:
model_lgb.fit(all_data[:1482535], y_train)
lgb_train_pred = model_lgb.predict(all_data[:1482535])
lgb_pred_1 = np.expm1(model_lgb.predict(all_data[1482535:]))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
model_lgb.score(all_data[:1482535], y_train)

In [ ]:
train["brand_name"] = train["brand_name"].fillna("None")

In [ ]:
train_mean = train.groupby('brand_name')['price'].mean().reset_index()
train_min = train.groupby('brand_name')['price'].min().reset_index()
train_max = train.groupby('brand_name')['price'].max().reset_index()
train_med = train.groupby('brand_name')['price'].median().reset_index()
train_cou = train.groupby('brand_name')['price'].count().reset_index()

In [ ]:
train_mean = train_mean.rename(columns={'price': 'price_mean'})
train_min = train_min.rename(columns={'price': 'price_min'})
train_max = train_max.rename(columns={'price': 'price_max'})
train_med = train_med.rename(columns={'price': 'price_med'})
train_cou = train_cou.rename(columns={'price': 'price_cou'})
col_1 =['brand_name']
col_2 =['brand_name']
train_min = train_min.drop(col_2, axis =1)
train_max = train_max.drop(col_1, axis =1)
train_med = train_med.drop(col_2, axis =1)
train_cou = train_cou.drop(col_2, axis =1)

In [ ]:
brand =  pd.concat([train_mean,train_min, train_max,train_med,train_cou], axis = 1)

In [ ]:
train["category_name"] = train["category_name"].fillna("None")
test["category_name"] = test["category_name"].fillna("None")

In [ ]:
splitted_date = train['category_name'].str.split('/', expand=True)
splitted_date.columns = ['cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5']
train = pd.concat([train,splitted_date ], axis = 1)

In [ ]:
splitted_date = test['category_name'].str.split('/', expand=True)
splitted_date.columns = ['cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5']
test = pd.concat([test,splitted_date ], axis = 1)

In [ ]:
train["cat_2"] = train["cat_2"].fillna("None")
train["cat_3"] = train["cat_3"].fillna("None")
train["cat_4"] = train["cat_4"].fillna("None")
train["cat_5"] = train["cat_5"].fillna("None")
test["cat_2"] = test["cat_2"].fillna("None")
test["cat_3"] = test["cat_3"].fillna("None")
test["cat_4"] = test["cat_4"].fillna("None")
test["cat_5"] = test["cat_5"].fillna("None")

In [ ]:
train['cate_f_1'] = train['category_name'].str.contains('Women')
train['cate_f_2'] = train['category_name'].str.contains('Men')
train['cate_f_3'] = train['category_name'].str.contains('Kids')
train['cate_f_4'] = train['category_name'].str.contains('Electronics')
train['cate_f_5'] = train['category_name'].str.contains('Jewelry')

In [ ]:
train['cate_f_1'] = train['cate_f_1'].replace([False,True],["0","1"])
train['cate_f_2'] = train['cate_f_2'].replace([False,True],["0","1"])
train['cate_f_3'] = train['cate_f_3'].replace([False,True],["0","1"])
train['cate_f_4'] = train['cate_f_4'].replace([False,True],["0","1"])
train['cate_f_5'] = train['cate_f_5'].replace([False,True],["0","1"])

In [ ]:
train['cate_f_f'] = train[['cate_f_1', 'cate_f_2','cate_f_3','cate_f_4','cate_f_5']].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
test['cate_f_1'] = test['category_name'].str.contains('Women')
test['cate_f_2'] = test['category_name'].str.contains('Men')
test['cate_f_3'] = test['category_name'].str.contains('Kids')
test['cate_f_4'] = test['category_name'].str.contains('Electronics')
test['cate_f_5'] = test['category_name'].str.contains('Jewelry')

In [ ]:
test['cate_f_1'] = test['cate_f_1'].replace([False,True],["0","1"])
test['cate_f_2'] = test['cate_f_2'].replace([False,True],["0","1"])
test['cate_f_3'] = test['cate_f_3'].replace([False,True],["0","1"])
test['cate_f_4'] = test['cate_f_4'].replace([False,True],["0","1"])
test['cate_f_5'] = test['cate_f_5'].replace([False,True],["0","1"])

In [ ]:
test['cate_f_f'] = test[['cate_f_1', 'cate_f_2','cate_f_3','cate_f_4','cate_f_5']].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
cat_cat_mean = train.groupby(['cate_f_f'])['price'].mean().reset_index()
cat_cat_mean = cat_cat_mean.rename(columns={'price': 'price_mean'})
cat_cat_min = train.groupby(['cate_f_f'])['price'].min().reset_index()
cat_cat_min = cat_cat_min.rename(columns={'price': 'price_min'})
col_1 =['cate_f_f']
cat_cat_min = cat_cat_min.drop(col_1, axis =1)
cat_cat_max = train.groupby(['cate_f_f'])['price'].max().reset_index()
cat_cat_max = cat_cat_max.rename(columns={'price': 'price_max'})
cat_cat_max = cat_cat_max.drop(col_1, axis =1)
cat_cat_med = train.groupby(['cate_f_f'])['price'].median().reset_index()
cat_cat_med = cat_cat_med.rename(columns={'price': 'price_med'})
cat_cat_med = cat_cat_med.drop(col_1, axis =1)
cat_st = pd.concat([cat_cat_mean,cat_cat_min,cat_cat_max, cat_cat_med], axis =1)

In [ ]:
train["item_description"] = train["item_description"].fillna("None")
test["item_description"] = test["item_description"].fillna("None")

In [ ]:
train['des_f_1'] = train['item_description'].str.contains('great')
train['des_f_2'] = train['item_description'].str.contains('rare')
train['des_f_3'] = train['item_description'].str.contains('heavy')
train['des_f_4'] = train['item_description'].str.contains('beautiful')
train['des_f_5'] = train['item_description'].str.contains('excellent')
train['des_f_6'] = train['item_description'].str.contains('amazing')
train['des_f_7'] = train['item_description'].str.contains('brand new')

In [ ]:
train['des_f_1'] = train['des_f_1'].replace([False,True],["0","1"])
train['des_f_2'] = train['des_f_2'].replace([False,True],["0","1"])
train['des_f_3'] = train['des_f_3'].replace([False,True],["0","1"])
train['des_f_4'] = train['des_f_4'].replace([False,True],["0","1"])
train['des_f_5'] = train['des_f_5'].replace([False,True],["0","1"])
train['des_f_6'] = train['des_f_6'].replace([False,True],["0","1"])
train['des_f_7'] = train['des_f_7'].replace([False,True],["0","1"])
train['des_f_f'] = train[['des_f_1', 'des_f_2','des_f_3','des_f_4','des_f_5','des_f_6','des_f_7']].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
test['des_f_1'] = test['item_description'].str.contains('great')
test['des_f_2'] = test['item_description'].str.contains('rare')
test['des_f_3'] = test['item_description'].str.contains('heavy')
test['des_f_4'] = test['item_description'].str.contains('beautiful')
test['des_f_5'] = test['item_description'].str.contains('excellent')
test['des_f_6'] = test['item_description'].str.contains('amazing')
test['des_f_7'] = test['item_description'].str.contains('brand new')

In [ ]:
test['des_f_1'] = test['des_f_1'].replace([False,True],["0","1"])
test['des_f_2'] = test['des_f_2'].replace([False,True],["0","1"])
test['des_f_3'] = test['des_f_3'].replace([False,True],["0","1"])
test['des_f_4'] = test['des_f_4'].replace([False,True],["0","1"])
test['des_f_5'] = test['des_f_5'].replace([False,True],["0","1"])
test['des_f_6'] = test['des_f_6'].replace([False,True],["0","1"])
test['des_f_7'] = test['des_f_7'].replace([False,True],["0","1"])
test['des_f_f'] = test[['des_f_1', 'des_f_2','des_f_3','des_f_4','des_f_5','des_f_6','des_f_7']].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
train_des_mean = train.groupby(['des_f_f'])['price'].mean().reset_index()
train_des_mean = train_des_mean.rename(columns={'price': 'price_mean_des'})
train_des_min = train.groupby(['des_f_f'])['price'].min().reset_index()
train_des_min = train_des_min.rename(columns={'price': 'price_min_des'})
col_1 =['des_f_f']
train_des_min = train_des_min.drop(col_1, axis =1)
train_des_max = train.groupby(['des_f_f'])['price'].max().reset_index()
train_des_max = train_des_max.rename(columns={'price': 'price_max_des'})
train_des_max = train_des_max.drop(col_1, axis =1)
train_des_med = train.groupby(['des_f_f'])['price'].median().reset_index()
train_des_med = train_des_med.rename(columns={'price': 'price_med_des'})
train_des_med = train_des_med.drop(col_1, axis =1)
des_st = pd.concat([train_des_mean,train_des_min,train_des_max, train_des_med], axis =1)

In [ ]:
train["price"] = np.log1p(train["price"])

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.price.values
all_data = pd.concat((train, test)).reset_index(drop = True)
all_data.drop(['price'], axis = 1, inplace = True)

In [ ]:
all_data_na = (all_data.isnull().sum()/len(all_data))*100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending = False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
all_data["brand_name"] = all_data["brand_name"].fillna("None")
all_data["category_name"] = all_data["category_name"].fillna("None")
all_data["item_description"] = all_data["item_description"].fillna("None")

In [ ]:
all_data = pd.merge(all_data, brand, on =('brand_name'), how='left')
all_data = pd.merge(all_data, cat_st, on =('cate_f_f'), how='left')
all_data = pd.merge(all_data, des_st, on =('des_f_f'), how='left')

In [ ]:
all_data['name_len'] = all_data['name'].str.len()
all_data['description_len'] =all_data['item_description'].str.len()

In [ ]:
all_data.head()

In [ ]:
drop_col = ['cate_f_f', 'des_f_f','cate_f_1', 'cate_f_2', 'cate_f_3', 'cate_f_4', 'cate_f_5', 'des_f_1', 'des_f_2', 'des_f_3', 'des_f_4', 'des_f_5', 'des_f_6', 'des_f_7', 'item_description','category_name','name']
all_data = all_data.drop(drop_col, axis = 1)

In [ ]:
cat_vars = ( 'brand_name', 'shipping',  'cat_1','cat_2','cat_3','cat_4','cat_5','cou_name')
#'item_condition_id',

In [ ]:
for v in cat_vars:
    all_data[v] = all_data[v].astype('category').cat.as_ordered()

In [ ]:
all_data.head()

In [ ]:
for v in cat_vars:
    all_data[v] = all_data[v].cat.codes

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import *

In [ ]:
lbl = preprocessing.LabelEncoder()

for c in cat_vars:
    lbl = LabelEncoder()
    lbl.fit(list(train[c].values))
    train[c] = lbl.transform(list(train[c].values))
    
for c in cat_vars:
    lbl = LabelEncoder()
    lbl.fit(list(test[c].values))
    test[c] = lbl.transform(list(test[c].values))

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred_2 = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
model_lgb.score(train, y_train)

In [ ]:
sub = lgb_pred_1*0.5 + lgb_pred_2*0.5

In [ ]:
df_result = pd.DataFrame()
df_result["test_id"] = test_id
df_result["price"] = sub
df_result.head()
df_result.to_csv("submit_1227_2.csv",index = False)